<a href="https://colab.research.google.com/github/Project-Jet-7/R.O.S.I.T.A/blob/main/ROSITA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

In [3]:
from transformers import pipeline

In [2]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Making the recording of the voice

This section is needed because there is a problem with the voice recording in the google colab. Colab does not allow using some libraries related to the real time voice recording. Hence, we had to improvise

reference: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be

NOTE: this code returns the voice IN BYTES

In [4]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
#from pydub import AudioSegment

In [5]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):

  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  #audio = AudioSegment.from_file(BytesIO(b))
  return b

# **Speech to text using WHISPER**

In [6]:
import os

In [7]:
def speech_to_text(whisper, voice_input):
  text = whisper(voice_input)
  return text['text']

testing

In [16]:
whisper = pipeline('automatic-speech-recognition', model='openai/whisper-small.en')


config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [20]:
# recording a voice and returning it IN BYTES
voice_input = record()
# recording it in separate file
with open('voice_input.mp3', mode='bx') as f:
  f.write(voice_input)

# using WHISPER to get intent (from speech to text)
intent = speech_to_text(whisper, voice_input)
print(intent)
# deleting the voice input (it is needed because we cannot record audio in the same file)
os.remove('voice_input.mp3')

<IPython.core.display.Javascript object>

 Rosita Howard


# **Loading fine-tunned BERT**


Using BERT to get the direction

In [10]:
def get_direction(BERT_classifier, text):

  command = BERT_classifier(text)[0]['label']
  return(command)

testing the function

In [ ]:
spoken_text = "plummet to the ground"
BERT_classifier = pipeline("text-classification", model="ROSITA-team/BERT_finetuned_for_directions")
direction = get_direction(BERT_classifier, spoken_text)

print(direction)

Go_down


# **Voice syntehis**


In [ ]:
!pip install --upgrade transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 32.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylin

In [11]:
import torch
from transformers import VitsTokenizer, VitsModel, set_seed
from IPython.display import Audio

In [12]:
def generate_voice_for_direction(voice_tokenizer, voice_model, direction):
  voice_text = "The direction you asked for is " + direction
  voice_inputs = voice_tokenizer(voice_text, return_tensors="pt")

  set_seed(500)

  with torch.no_grad():
   outputs = voice_model(**voice_inputs)

  waveform = outputs.waveform[0]

  return Audio(waveform, rate=voice_model.config.sampling_rate, autoplay=True)


testing the voice generation

In [ ]:
voice_tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-eng")
voice_model = VitsModel.from_pretrained("facebook/mms-tts-eng")

generate_voice_for_direction(voice_tokenizer, voice_model, direction)

Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

# Functions for making the voice assistant sound more human


In [13]:
def play_greeting(voice_tokenizer, voice_model):
  greeting = """ Hello, pilot! I am the first version of the ROSITA voice assistant.
  I am not ready yet and I cannot have a full dialogue with you. However,
  I know how to identify directions. Give me a command related to the direction
  and I will classify it. Please, press any button on the keyboard to start recording"""

  greeting_input = voice_tokenizer(greeting, return_tensors="pt")

  set_seed(555)

  with torch.no_grad():
   outputs = voice_model(**greeting_input)

  waveform = outputs.waveform[0]

  return Audio(waveform, rate=voice_model.config.sampling_rate, autoplay=True)





In [ ]:
play_greeting(voice_tokenizer, voice_model)

testing the function

# Main body

In [14]:
if __name__ == "__main__":

  # getting classifiers and tokenizers
  voice_tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-eng")
  voice_model = VitsModel.from_pretrained("facebook/mms-tts-eng")
  BERT_classifier = pipeline("text-classification", model="ROSITA-team/BERT_finetuned_for_directions")
  whisper = pipeline('automatic-speech-recognition', model='openai/whisper-medium.en')

  # Greeting the pilot
  display(play_greeting(voice_tokenizer, voice_model))

  start = input("press any button on keybord to start recording")

  # recording a voice and returning it IN BYTES
  voice_input = record()
  # recording it in separate file
  with open('voice_input.mp3', mode='bx') as f:
    f.write(voice_input)

  # using WHISPER to get intent (from speech to text)
  intent = speech_to_text(whisper, voice_input)
  print("intent is " + intent)

  # deleting the voice input (it is needed because we cannot record audio in the same file)
  os.remove('voice_input.mp3')

  # classifying the direction
  direction = get_direction(BERT_classifier, intent)
  display(generate_voice_for_direction(voice_tokenizer, voice_model, direction))
  print("direction classifying is done")




tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

press any button on keybord to start recordingn


<IPython.core.display.Javascript object>

intent is  Zeta, go left.


direction classifying is done


In [ ]:
# !pip uninstall torch
# !pip install torch==2.2.2+cu118 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu118

# !apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
# !pip install PyAudio
# !pip install pyaudio
# !pip install RealtimeSTT

# https://github.com/KoljaB/RealtimeSTT
from RealtimeSTT import AudioToTextRecorder
# # manual
# recorder.start()
# recorder.stop()
# print(recorder.text())
# #automatic
with AudioToTextRecorder() as recorder:
    print(recorder.text())
# #apply
# def process_text(text):
#     print (text)
# while True:
#     recorder.text(process_text)

# # wake word
# recorder = AudioToTextRecorder(wake_words="jarvis")

# print('Say "Jarvis" then speak.')
# print(recorder.text())

ModuleNotFoundError: No module named 'RealtimeSTT'